# Study case: Brain vasculature in acute ischemic stroke

##### License: Apache 2.0

In [1]:
# Load giotto packages
import sys
sys.path.insert(0, './giotto-tda')
from gtda.images import Binarizer, Inverter, HeightFiltration, RadialFiltration
from gtda.homology import VietorisRipsPersistence, CubicalPersistence
from gtda.diagrams import  PairwiseDistance, Amplitude, Scaler, PersistenceEntropy, BettiCurve, PersistenceLandscape, HeatKernel
from BnDs.analysis.utils.ImageToPointCloud import ImageToPointCloud

print('Successfully loaded giotto!')


Successfully loaded giotto!


In [61]:
# Load BnDs specific functions
from BnDs.analysis import data_loader as dl

In [62]:
# Load generic packages
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion, make_union
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.base import clone

import itertools
import numpy as np
import pandas as pd
import random
import gzip
import pickle as pkl
import itertools
import matplotlib as mpl
import matplotlib.pyplot as plt
#%matplotlib inline

## Loading the stroke dataset

In [63]:
# Read the data
data_dir = './data/'
clinical_inputs, ct_inputs, ct_lesion_GT, mri_inputs, mri_lesion_GT, brain_masks, ids, params = dl.load_structured_data(data_dir, 'data_set.npz')
lesion_volumes = np.sum(ct_lesion_GT, axis=(1, 2, 3))

n_subj, n_x, n_y, n_z, n_channels = ct_inputs.shape
X = np.squeeze(ct_inputs)
y = lesion_volumes

X = X[:10]
y = y[:10]

print(X.shape, y.shape)
print(np.min(X), np.max(X))

Loading a total of 113 subjects.
Sequences used: {'ct_sequences': ['wmask_filtered_extracted_betted_Angio'], 'ct_label_sequences': ['wcoreg_VOI'], 'mri_sequences': [], 'mri_label_sequences': []}
0 subjects had been excluded.
(10, 79, 95, 79) (10,)
0.0 1.0


In [64]:
# Set up the data
n_train, n_test = int(X.shape[0] * 0.8), int(X.shape[0] * 0.2)

X_train = X[:n_train]
y_train = y[:n_train]
X_test = X[n_train:n_train+n_test]
y_test = y[n_train:n_train+n_test]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(8, 79, 95, 79) (8,) (2, 79, 95, 79) (2,)


In [65]:
# Create directions and centers along every dimension
direction_list = [list(p) for p in itertools.product([1, 0, -1], repeat=3)]
direction_list.remove([0, 0, 0])
center_per_dim = [[int(0.25*n_x), int(0.5*n_x), int(0.75*n_x)], 
                  [int(0.25*n_y), int(0.5*n_y), int(0.75*n_y)], 
                  [int(0.25*n_z), int(0.5*n_z), int(0.75*n_z)]]
center_list = [list(p) for p in itertools.product(*center_per_dim)]


filtration_list = \
 [HeightFiltration(direction=direction) 
   for direction in direction_list] + \
 [RadialFiltration(center=center) 
   for center in center_list] + \
 [None]

binarizer = Binarizer(threshold=0.4)
point = ImageToPointCloud()
cubical = CubicalPersistence(homology_dimensions=[0, 1])
rips = VietorisRipsPersistence(homology_dimensions=[0, 1])
scaler = Scaler(metric='bottleneck')

grayscale_steps = [[cubical, scaler]]
filtration_steps = [[binarizer, filtration, cubical, scaler] for filtration in filtration_list]
# Point step provokes:
# ValueError: Input contains NaN, infinity or a value too large for dtype('float64').
# Cause: Point gives infinity for all values where there is no point in order to conserve the dimensions of the array
# Temporary solution: remove check_array from VietorisRips function responsible for checking for Inf
rips_steps = [[binarizer, point, rips, scaler]]

image_steps = grayscale_steps + filtration_steps + rips_steps

metric_list = [ 
   {'metric': 'bottleneck', 'metric_params': {'p': np.inf}},
   {'metric': 'wasserstein', 'metric_params': {'p': 1}},
   {'metric': 'wasserstein', 'metric_params': {'p': 2}},
   {'metric': 'landscape', 'metric_params': {'p': 1, 'n_layers': 1, 'n_values': 100}},
   {'metric': 'landscape', 'metric_params': {'p': 1, 'n_layers': 2, 'n_values': 100}},
   {'metric': 'landscape', 'metric_params': {'p': 2, 'n_layers': 1, 'n_values': 100}},
   {'metric': 'landscape', 'metric_params': {'p': 2, 'n_layers': 2, 'n_values': 100}},
   {'metric': 'betti', 'metric_params': {'p': 1, 'n_values': 100}},
   {'metric': 'betti', 'metric_params': {'p': 2, 'n_values': 100}},
   {'metric': 'heat', 'metric_params': {'p': 1, 'sigma': 1.6, 'n_values': 100}},
   {'metric': 'heat', 'metric_params': {'p': 1, 'sigma': 3.2, 'n_values': 100}},
   {'metric': 'heat', 'metric_params': {'p': 2, 'sigma': 1.6, 'n_values': 100}},
   {'metric': 'heat', 'metric_params': {'p': 2, 'sigma': 3.2, 'n_values': 100}}
]

entropy_steps = [steps + [PersistenceEntropy()] for steps in image_steps]
amplitude_steps = [steps+[Amplitude(**metric, order=None)] for steps in image_steps for metric in metric_list]

all_steps = entropy_steps + amplitude_steps
feature_union = make_union(*[make_pipeline(*steps) for steps in all_steps], n_jobs=-1)

In [ ]:
X_train_tda = feature_union.fit_transform(X_train, y_train)
pkl.dump(X_train_tda, open('X_train_tda.pkl', 'wb'))

In [ ]:
X_test_tda = feature_union.transform(X_test)
pkl.dump(X_test_tda, open('X_test_tda.pkl', 'wb'))

In [ ]:
print(X_train_tda.shape)

In [ ]:
classifier = RandomForestClassifier(n_estimators=10000, n_jobs=-1)
classifier.fit(X_train_tda, y_train)

In [ ]:
y_train_predict = classifier.predict(X_train_tda)
pkl.dump(y_train_predict, open('y_train_predict_tda.pkl', 'wb'))

In [ ]:
y_test_predict = classifier.predict(X_test_tda)
pkl.dump(y_test_predict, open('y_test_predict_tda.pkl', 'wb'))

In [ ]:
confusion = confusion_matrix(y_test, y_test_predict)
plt.imshow(confusion)
plt.show()

In [ ]:
importance = -classifier.feature_importances_
importance_indices = np.argsort(importance)

In [ ]:
X_train_importance = X_train_tda[:, importance_indices]
print(X_train_importance.shape)

In [ ]:
correlation = np.abs(np.corrcoef(X_train_tda.T))
plt.imshow(correlation)
plt.show()

In [ ]:
def select_features(correlation_, importance_indices_, threshold_):
    decorrelated_index = importance_indices_[:1]

    for index in importance_indices_[1:]:
        if np.sum(correlation_[decorrelated_index, index] > threshold_) == 0:
            decorrelated_index = np.append(decorrelated_index, [index])
    return decorrelated_index

In [ ]:
def run_cv_scores(thresholds_list_, n_features_list_, correlation_, importance_indices_, postfix, n_folds=3):
    for threshold_ in thresholds_list_:
        print('threshold: ', threshold_)
        decorrelated_feature_index = select_features(correlation_, importance_indices_, threshold_=threshold_)
        n_features_max = decorrelated_feature_index.shape[0]
        print('Number of decorrelated features: ', n_features_max)
        n_features_list = sorted(list(set([n_features if n_features <= n_features_max else n_features_max
                               for n_features in n_features_list_])))
        print('n_features_list: ', n_features_list)

        cv_scores = {}
        cv = KFold(n_folds)
        for n_features in n_features_list:
            X_train_n_features = X_train_tda[:, decorrelated_feature_index[:n_features]]
            print('X_train shape: ', X_train_n_features.shape)
            cv_scores[n_features] = cross_val_score(classifier, 
                                                    X_train_n_features, 
                                                    y_train, cv=cv)
            print('cv scores for', postfix, ': ', n_features, cv_scores[n_features])
        pkl.dump(cv_scores, open('cv_scores_'+str(threshold_)+'_'+postfix+'.pkl', 'wb'))

In [ ]:
thresholds_list = [0.8, 0.9, 0.95, 1.0]
n_features_list = [4, 8, 28, 42, 56, 112, 178, 244, 392, 784]
run_cv_scores(thresholds_list, n_features_list, correlation, importance_indices, 'tda')